## [Finding a Protein Motif]()

### Background
A structural and functional unit of the protein is a **protein domain**: in terms of the protein's primary structure, the domain is an interval of amino acids that can evolve and function independently.

Just like species, proteins can evolve, forming homologous groups called protein families. Proteins from one family usually have the same set of domains, performing similar functions; see Figure 1.

A component of a domain essential for its function is called a **motif**, a term that in general has the same meaning as it does in nucleic acids, although many other terms are also used (blocks, signatures, fingerprints, etc.) Usually protein motifs are evolutionarily conservative, meaning that they appear without much change in different species.

Proteins are identified in different labs around the world and gathered into freely accessible databases. A central repository for protein data is **[UniProt](https://www.uniprot.org/)**, which provides detailed protein annotation, including function description, domain structure, and post-translational modifications. UniProt also supports protein similarity search, taxonomy analysis, and literature citations.

### UniProt
To allow for the presence of its varying forms, a protein motif is represented by a shorthand as follows: [XY] means "either X or Y" and {X} means "any amino acid except X." For example, the N-glycosylation motif is written as N{P}[ST]{P}.

You can see the complete description and features of a particular protein by its access ID "uniprot_id" in the UniProt database, by inserting the ID number into
```
http://www.uniprot.org/uniprot/uniprot_id
```

Alternatively, you can obtain a protein sequence in FASTA format by following
```
http://www.uniprot.org/uniprot/uniprot_id.fasta
```

For example, the data for protein B5ZC00 can be found at http://www.uniprot.org/uniprot/B5ZC00.

### Problem
**Given:** At most 15 UniProt Protein Database access IDs.

**Return:** For each protein possessing the N-glycosylation motif, output its given access ID followed by a list of locations in the protein string where the motif can be found.

### Example
Input:
```
A2Z669
B5ZC00
P07204_TRBM_HUMAN
P20840_SAG1_YEAST
```

Output:
```
B5ZC00
85 118 142 306 395
P07204_TRBM_HUMAN
47 115 116 382 409
P20840_SAG1_YEAST
79 109 135 248 306 348 364 402 485 501 614
```

In [59]:
import requests
from wasims_toolbox import read_fasta

def get_protein_sequence(access_id: str) -> dict:
    
    """
    Returns protein sequence from given UniProt access ID
    as dictionary with key-value pairs of name to sequence.
    
    Args:
        access_id (str): UniProt Protein Database access ID
    
    Returns:
        dict: Protein name and sequence
    """
    
    response = requests.get(f"https://rest.uniprot.org/uniprotkb/{access_id}.fasta")
    protein = read_fasta(sequences=response.text)
    return protein

proteins = {}
access_ids = ["B5ZC00", "P07204", "P20840"]

for access_id in access_ids:
    protein = get_protein_sequence(access_id)
    proteins.update(protein)

proteins

{'sp|B5ZC00|SYG_UREU1 Glycine--tRNA ligase OS=Ureaplasma urealyticum serovar 10 (strain ATCC 33699 / Western) OX=565575 GN=glyQS PE=3 SV=1': 'MKNKFKTQEELVNHLKTVGFVFANSEIYNGLANAWDYGPLGVLLKNNLKNLWWKEFVTKQKDVVGLDSAIILNPLVWKASGHLDNFSDPLIDCKNCKARYRADKLIESFDENIHIAENSSNEEFAKVLNDYEISCPTCKQFNWTEIRHFNLMFKTYQGVIEDAKNVVYLRPETAQGIFVNFKNVQRSMRLHLPFGIAQIGKSFRNEITPGNFIFRTREFEQMEIEFFLKEESAYDIFDKYLNQIENWLVSACGLSLNNLRKHEHPKEELSHYSKKTIDFEYNFLHGFSELYGIAYRTNYDLSVHMNLSKKDLTYFDEQTKEKYVPHVIEPSVGVERLLYAILTEATFIEKLENDDERILMDLKYDLAPYKIAVMPLVNKLKDKAEEIYGKILDLNISATFDNSGSIGKRYRRQDAIGTIYCLTIDFDSLDDQQDPSFTIRERNSMAQKRIKLSELPLYLNQKAHEDFQRQCQK',
 'sp|P07204|TRBM_HUMAN Thrombomodulin OS=Homo sapiens OX=9606 GN=THBD PE=1 SV=2': 'MLGVLVLGALALAGLGFPAPAEPQPGGSQCVEHDCFALYPGPATFLNASQICDGLRGHLMTVRSSVAADVISLLLNGDGGVGRRRLWIGLQLPPGCGDPKRLGPLRGFQWVTGDNNTSYSRWARLDLNGAPLCGPLCVAVSAAEATVPSEPIWEEQQCEVKADGFLCEFHFPATCRPLAVEPGAAAAAVSITYGTPFAARGADFQALPVGSSAAVAPLGLQLMCTAPPGAVQGHWAREAPGAWDCSVENGGCEHACNAIPGAPRCQCPAGAALQADGRSCTASATQSCNDLCEHFCVPN

In [60]:
def find_n_glycosylation_motifs(protein: str) -> list:
    
    """
    Returns locations of N-glycosylation motif for
    the given protein using UniProt access ID.
    
    Args:
        protein (str): Protein sequence
    
    Returns:
        list: Positions in the protein chain where the 
        N-glycosylation motif occurs.
    """
    
    locations = []
    
    amino_acids = [
        'A', 'C', 'D', 'E', 'F', 'G', 'H', 
        'I', 'K', 'L', 'M', 'N', 'Q', 'R', 
        'S', 'T', 'V', 'W', 'Y'
    ]
    
    motifs = [f"N{aa1}S{aa2}" for aa1 in amino_acids for aa2 in amino_acids]
    motifs += [f"N{aa1}T{aa2}" for aa1 in amino_acids for aa2 in amino_acids]
    
    for motif in motifs:
        locations += [i+1 for i in range(len(protein)) if protein.startswith(motif, i)]
        
    return sorted(locations)

In [61]:
def print_solution(access_ids: list, proteins: dict):
    """Print solution to Rosalind problem."""
    
    for index, protein in enumerate(proteins.values()):
        motif_locations = find_n_glycosylation_motifs(protein)
        motif_locations = [str(loc) for loc in motif_locations]
        print(access_ids[index])
        print(str.join(" ", motif_locations))

print_solution(access_ids, proteins)

B5ZC00
85 118 142 306 395
P07204
47 115 116 382 409
P20840
79 109 135 248 306 348 364 402 485 501 614


In [64]:
access_ids = [
    "A6NM15",
    "Q58CQ5",
    "P00744_PRTZ_BOVIN",
    "Q7TMB8",
    "P11279_LMP1_HUMAN",
    "P98119_URT1_DESRO",
    "Q6A9W5",
    "P01044_KNH1_BOVIN",
    "B5ZC00",
    "P49286",
    "P07204_TRBM_HUMAN",
    "P01374_TNFB_HUMAN"
]

proteins = {}
for access_id in access_ids:
    protein = get_protein_sequence(access_id.split("_")[0])
    proteins.update(protein)
    print(protein)

IndexError: list index out of range

In [63]:
print_solution(access_ids, proteins)

Q83I57

P25174
17 32 56 97 116 132 151 178 183 198 325 670
Q8ZRE7

P01880_DTC_HUMAN
225 316 367
Q8CE94
369
P27918_PROP_HUMAN
428
Q50228
55 228
Q3B391
238
A1USX4

Q0U2C0
43 330
P10761_ZP3_MOUSE
146 273 304 327 330
P05231_IL6_HUMAN
73 172
P05113_IL5_HUMAN
47 90
P14210_HGF_HUMAN
294 402 566 653
